Baseline models - consisting of Linear Regression, RFRegressor, XGBoostRegressor fed into a linear regression ensemble. 

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import xgboost as xg 
import DecisionTreeRegressor as dtg

In [81]:
df_news = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/final_sample_finbert.csv')
columns_to_select = ['Date_hourly', 'weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']
df_news = df_news[columns_to_select]
df_news['Date_hourly'] = df_news['Date_hourly'].astype(str).str.strip()

In [82]:
df_stock_price = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/us_2024_ticker combined versions/SPY_Combined_WithPriceChange.csv',index_col=0)
df_stock_price
df_stock_price['Date'] = df_stock_price['Date'].astype(str)
df_stock_price['Time'] = df_stock_price['Time'].astype(str).str.zfill(6)

df_stock_price['Date_hourly'] = '20' + df_stock_price['Date'] + df_stock_price['Time'].str[:2]
df_stock_price=df_stock_price.iloc[:, -2:]

In [4]:
# Merge the datasets on the 'Date_hourly' column using an inner join to keep only matching rows
merged_df = pd.merge(df_news, df_stock_price, on='Date_hourly', how='inner')
merged_df.sort_values("weighted_negative_fb",ascending=False)

NameError: name 'df_news' is not defined

In [84]:
# # Train Test Split and creating of x_train, x_val, x and y_train, y_val, y_test
# Y = merged_df[['PriceChange']]  
# X = merged_df[['weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']]  
# x_train_val, x_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.2, random_state=1010)
# x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=1010)  

In [5]:
merged_df = pd.read_csv('/Users/oryining/Documents/NUS MComp/Semester 2/CS5344 - Big Data Analytics Technology/CS5344_Project/1. Data Retrieval and Processing of News and Stock Prices/final_dataset.csv',index_col=0)

# Train Test Split and creating of x_train, x_val, x and y_train, y_val, y_test
Y = merged_df[['Pct_Change']]  
X = merged_df[['weighted_positive_fb', 'weighted_negative_fb', 'weighted_neutral_fb','weighted_DocTone']]  
x_train_val, x_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.2, random_state=1616)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=1616)  

Linear Regression in baseline form. 

In [6]:
def LinearRegressionBaseLine(x_train,x_test,y_train,y_test):
    
    #Building model with Train data
    reg_model= LinearRegression().fit(x_train, y_train)
    y_pred = reg_model.predict(x_test)
    val_rmse_reg_model = root_mean_squared_error(y_test, y_pred)
    r2_reg_model = r2_score(y_test,y_pred)
    
    print(f'Test RMSE: {val_rmse_reg_model}')
    print(f'Test R2: {r2_reg_model}')

    print("Model details are coefficients of "+str(reg_model.coef_)+" intercept of "+str(reg_model.intercept_))

    return y_pred,reg_model


In [7]:
LinearRegressionBaseLine(x_train,x_test,y_train,y_test)

Test RMSE: 1.648817326439581
Test R2: 0.6508259456420558
Model details are coefficients of [[ 4.22366227 -2.62350391  0.80944256 -0.11299289]] intercept of [-0.71761997]


(array([[-1.5039925 ],
        [ 1.00041905],
        [-1.59979466],
        ...,
        [-2.03336589],
        [ 2.84563284],
        [-2.17758208]]),
 LinearRegression())

Random Forest Regressor in baseline form

In [8]:
# TODO : Incoporating K Fold Cross Validation to Further Tune

In [9]:
def RandomForestRegressorBaseline(x_train,x_test,y_train,y_test):
    
    rf_model = RandomForestRegressor(random_state=30)
    rf_model.fit(x_train, y_train.values.ravel())
    y_pred = rf_model.predict(x_test)
    val_rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Test RMSE: {val_rmse}')
    print(f'Test R2: {r2}')

    return y_pred,rf_model

In [10]:
RandomForestRegressorBaseline(x_train,x_test,y_train,y_test)

Test RMSE: 1.4604754895799814
Test R2: 0.7260410921341554


(array([-2.52151805,  2.66661032, -3.18558855, ..., -1.81774796,
         2.54803464, -2.54003689]),
 RandomForestRegressor(random_state=30))

XGBoost Regressor in baseline form 

In [11]:
def XGBoostRegressorBaseline(x_train,x_test,y_train,y_test):
    
    xgb_model = xg.XGBRegressor(random_state=42)
    xgb_model.fit(x_train, y_train.values.ravel())
    y_pred = xgb_model.predict(x_test)
    val_rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Test RMSE: {val_rmse}')
    print(f'Test R2: {r2}')

    return y_pred,xgb_model

Decision Tree Baseline Form

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def DecisionTreeRegressorBaseline(x_train, x_test, y_train, y_test):
    # Ensure y_train and y_test are 1D arrays for compatibility
    y_train = np.ravel(y_train)  # Converts y_train to a 1D array if it isn't already
    y_test = np.ravel(y_test)    # Converts y_test to a 1D array if it isn't already

    # Initialize and fit the model
    dt_model = DecisionTreeRegressor()
    dt_model.fit(x_train, y_train)

    # Make predictions on test data
    y_pred_test = dt_model.predict(x_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_r2 = r2_score(y_test, y_pred_test)

    # Make predictions on training data
    y_pred_train = dt_model.predict(x_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    train_r2 = r2_score(y_train, y_pred_train)

    # Print metrics
    print(f'Test RMSE: {test_rmse}')
    print(f'Test R2: {test_r2}')
    print(f'Train RMSE: {train_rmse}')
    print(f'Train R2: {train_r2}')

    return y_pred_test, dt_model


In [13]:
DecisionTreeRegressorBaseline(x_train,x_test,y_train,y_test)

Test RMSE: 1.9192529069482223
Test R2: 0.5268909074373522
Train RMSE: 0.0
Train R2: 1.0


(array([-1.43178846,  2.44032979, -3.20240389, ..., -0.1550891 ,
         3.53855464, -0.99602472]),
 DecisionTreeRegressor())

Ensemble model taking in variables from 3 individual models to combine outputs into a linear regression model

In [13]:
def EnsembleBaseLine():

    #This section calls the initial models to return a series of predictions
    print("\n")
    print("===============Linear Regresion BaseLine Outputs=====================")
    y_pred_linear_regression_input, input_reg_model= LinearRegressionBaseLine(x_train,x_test,y_train,y_test)
    print("===============RandomForest Regressor Baseline Outputs================")
    y_pred_random_forest_regressor,input_rf_model = RandomForestRegressorBaseline(x_train,x_test,y_train,y_test)
    print("===============XGBoost Regressor Baseline Outputs================")
    y_pred_xgb_regressor,input_xgb_model = XGBoostRegressorBaseline(x_train,x_test,y_train,y_test)
    print("===============Decision Tree Regressor Baseline Outputs================")
    y_pred_dt_regressor,input_dt_model = DecisionTreeRegressorBaseline(x_train,x_test,y_train,y_test)

    #Model predictions are combined to be passed into the ensemble
    y_pred_stacked_predictions = np.column_stack((y_pred_linear_regression_input, y_pred_random_forest_regressor,y_pred_xgb_regressor,y_pred_dt_regressor))
    y_pred_stacked_predictions_df = pd.DataFrame(y_pred_stacked_predictions, columns=['reg_pred1', 'rf_pred2','xgboost_pred3','dt_pred4'])

    #Processing of data to be passed in as ensemble prediction using x_val 
    y_val_input_reg_model = input_reg_model.predict(x_val)
    y_val_input_rf_model = input_rf_model.predict(x_val)
    y_val_input_xgb_model = input_xgb_model.predict(x_val)
    y_val_input_dt_model = input_dt_model.predict(x_val)

    y_val_input_model_predictions = np.column_stack((y_val_input_reg_model, y_val_input_rf_model,y_val_input_xgb_model, y_val_input_dt_model))
    y_val_input_model_predictions_df = pd.DataFrame(y_val_input_model_predictions, columns=['reg_pred1', 'rf_pred2','xgboost_pred3','dt_pred4'])
    
    
    #Ensemble Model 
    print("\n")
    print("===============Ensemble Baseline Model============================")
    ensemble_model= LinearRegression().fit(y_pred_stacked_predictions_df,y_test) # Predicted Values of the Models, and y_test is the Y variable
    y_ensemble = ensemble_model.predict(y_val_input_model_predictions_df) 

    val_rmse = root_mean_squared_error(y_ensemble, y_val)
    r2 = r2_score(y_ensemble, y_val)
    print(f'Validation RMSE: {val_rmse}')
    print(f'Validation R2: {r2}')

    print("Model details are coefficients of "+str(ensemble_model.coef_)+" intercept of "+str(ensemble_model.intercept_))

    # print(y_ensemble)
    # print(y_val)

In [14]:
EnsembleBaseLine()



===============Linear Regresion BaseLine Outputs=====================
Test RMSE: 1.648817326439581
Test R2: 0.6508259456420558
Model details are coefficients of [[ 4.22366227 -2.62350391  0.80944256 -0.11299289]] intercept of [-0.71761997]
===============RandomForest Regressor Baseline Outputs================
Test RMSE: 1.4604754895799814
Test R2: 0.7260410921341554
===============XGBoost Regressor Baseline Outputs================
Test RMSE: 1.5011383343942033
Test R2: 0.7105734944343567
===============Decision Tree Regressor Baseline Outputs================
Test RMSE: 1.939963171522729
Test R2: 0.5166253716076841


===============Ensemble Baseline Model============================
Validation RMSE: 1.4069870815370709
Validation R2: 0.6486742363384139
Model details are coefficients of [[1.85856476e-01 6.29047600e-01 2.17854312e-01 3.03612031e-04]] intercept of [-0.02242535]
